In [1]:
import pandas as pd
import re
from datetime import datetime as dt

YEAR_RANGE = (1900, 1950)

def remove(row):
    try:
        date = dt.strptime(row.Date, '%Y-%m-%d')
    except:
        return True
    
    if YEAR_RANGE and date.year >= YEAR_RANGE[0] and date.year <= YEAR_RANGE[1]:
        pass # in the range!
    else:
        return True

    if row['Exclude from visualization'] or row['Unsure whether drag artist']:
        return True
    
    no_city = row['City'] == ''
    no_performer = row['Performer'] == ''
    no_venue = row['Venue'] == ''
    unnamed_performer = 'unnamed' in row['Performer'].lower()
    
    if no_city and no_performer and no_venue:
        return True
    
    if unnamed_performer:
        return True

    return False

def extract_addresses_dict(normalized_df):
    addresses = {}
    rows_with_addresses = normalized_df[normalized_df['Address']!='']
    warnings = []
    for x in zip(rows_with_addresses['Date'], rows_with_addresses['Source'], rows_with_addresses['Venue'], rows_with_addresses['Address']):
        date, source, venue, address = x
        if venue == '':
            warnings.append(address)
        else:
            if not venue in addresses:
                addresses[venue] = {}
            if not source in addresses[venue]:
                addresses[venue][source] = address
    if len(warnings):
        print(f'Warning: {len(warnings)} Venues with no names have addresses:')
        print('- ' + '- '.join(warnings))

    return addresses

def reverse_comment_dict(comment_dict):
    comments_reverse = {}
    for performer, comments in comment_dict.items():
        if not performer in comments_reverse:
            comments_reverse[performer] = {}
        for source, comment in comments.items():
            if not comment in comments_reverse[performer]:
                comments_reverse[performer][comment] = []
            comments_reverse[performer][comment].append(source)
    return comments_reverse
        
def get_comments(df, comment_field='Comment on edge: revue', match_field='Revue', transform=None):
    comments = {}
    rows_with_comments = df[df[comment_field]!='']
    warnings = []
    for x in zip(rows_with_comments['Date'], rows_with_comments['Source'], rows_with_comments[match_field], rows_with_comments[comment_field]):
        date, source, match, comment = x
        comment = str(comment).strip()
        if transform:
            comment = transform(comment)
        if match == '':
            warnings.append(str(comment)[:40]+'...')
        else:
            if not match in comments:
                comments[match] = {}
            if not source in comments[match]:
                comments[match][source] = comment
    if len(warnings):
        print(f'Warning: {len(warnings)} mentions in `{comment_field}` with no value have comments:')
        print('- ' + '\n- '.join(warnings))

    return comments

def get_revue_comments_dict(df):
    return get_comments(df, 'Comment on edge: revue', 'Revue')

def get_performer_comments_dict(df):
    return get_comments(df, 'Comment on node: performer', 'Performer')

def get_venue_comments_dict(df):
    return get_comments(df, 'Comment on node: venue', 'Venue')

def get_city_comments_dict(df):
    return get_comments(df, 'Comment on node: city', 'City')

def get_true_value(row, type):
    if type == 'source':
        if row['Source clean'] != '':
            return row['Source clean']
        return row['Source']
    if type == 'performer':
        if row['Normalized performer'] != '':
            return row['Normalized performer']
        if row['Performer first-name'] != '' and row['Performer last-name'] != '':
            return row['Normalized performer']
        return row['Performer']
    if type == 'city':
        if row['Normalized City'] != '':
            return row['Normalized City']
        return row['City']
    if type == 'revue':
        if row['Normalized Revue Name'] != '':
            return row['Normalized Revue Name']
        return row['Revue name']
    if type == 'venue':
        if row['Normalized Venue'] != '':
            return row['Normalized Venue']
        return row['Venue']
    raise NotImplementedError(f'type `{type}` is not yet implemented')

def find_ref(row, eima=True):
    source = row['Source']
    source += ' ' + row['EIMA']
    source += ' ' + row['Search (newspapers.com)']
    source += ' ' + row['Source clean']
    
    is_eima = 'eima' in source.lower() or 'variety' in source.lower() or 'billboard' in source.lower()
    has_ref = re.search(r'(\d{7,10})', source)
    refs = list(set(re.findall(r'(\d{7,10})', source)))
    if has_ref and eima and is_eima:
        return '|'.join(refs)
    
    if has_ref and not eima and not is_eima:
        return '|'.join(refs)

    return ''
    

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT0E0Y7txIa2pfBuusA1cd8X5OVhQ_D0qZC8D40KhTU3xB7McsPR2kuB7GH6ncmNT3nfjEYGbscOPp0/pub?gid=0&single=true&output=csv')
df = df.fillna('')
df = df.replace('—', '')
df = df.replace('—*', '')


In [2]:
# First, set up our references to EIMA and newspapers.com
df['EIMA'] = df.apply(lambda row: find_ref(row), axis=1)
df['Newspapers.com'] = df.apply(lambda row: find_ref(row, False), axis=1)

# Normalize dataframe
df['Source'] = df.apply(lambda row: get_true_value(row, 'source'), axis=1)
df['Venue'] = df.apply(lambda row: get_true_value(row, 'venue'), axis=1)
df['Performer'] = df.apply(lambda row: get_true_value(row, 'performer'), axis=1)
df['City'] = df.apply(lambda row: get_true_value(row, 'city'), axis=1)
df['Revue'] = df.apply(lambda row: get_true_value(row, 'revue'), axis=1)


In [3]:
# Drop filtered data
df['remove'] = df.apply(lambda row: remove(row), axis=1)
df = df.drop(df[df['remove']==True].index)

In [4]:
sources = list(set([x for x in df['Source']]))

In [5]:
ID_NUMBERS = re.compile(r'\d{6,10}')
DATES = re.compile(r'(January|February|March|April|May|June|July|August|September|October|November|December) (\d{1,2}), (\d{4})')
ENDS_WITH_PAGE = re.compile(r', (\d{1,2})$')


In [6]:
sources_without_id = [x for x in sources if not ID_NUMBERS.search(x)]

sources_with_date = [x for x in sources_without_id if DATES.search(x)]

sources_with_date = sorted(sources_with_date)


papers = [DATES.split(x)[0][:-2] for x in sources_with_date]
dates = [DATES.search(x).groups() for x in sources_with_date]
# dates = [(x[2], x[0].replace('January', '01').replace('February', '02').replace('March', '03').replace('April', '04').replace('May', '05').replace('June', '06').replace('July', '07').replace('August', '08').replace('September', '09').replace('October', '10').replace('November', '11').replace('December', '12'), f'{int(x[1]):02d}') for x in dates]
dates = [(x[2], x[0], x[1]) for x in dates]

In [7]:

clear_page_number = [ENDS_WITH_PAGE.search(x) for x in sources_with_date]

In [8]:
source_data_file = 'sources-data.json'

In [9]:
from selenium.common.exceptions import NoSuchElementException

In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

try:
    b.get('http://www.newspapers.com')
except:
    b = webdriver.Firefox()
    b.get('http://www.newspapers.com')

In [11]:
import json, time
from pathlib import Path

In [ ]:
LIMIT = 4000

def get_current_variables():
    try:
        return json.loads(Path('data.json').read_text())
    except FileNotFoundError:
        print('could not load JSON so returning empty')
        return {
            'not_found': [],
            'firstpage_urls': {},
            'newspaper_urls': {},
            'page_urls': {},
            'ambiguous': [],
            'no_link': {}
        }
    
def set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link):
    data = {
        'not_found': not_found,
        'firstpage_urls': firstpage_urls,
        'newspaper_urls': newspaper_urls,
        'page_urls': page_urls,
        'ambiguous': ambiguous,
        'no_link': no_link
    }
    Path('data.json').write_text(json.dumps(data))


count = len([x for x in zip(papers, dates, clear_page_number)])
counter = 0

for newspaper, date, page in [x for x in zip(papers, dates, clear_page_number)][:LIMIT]:
    counter += 1

    not_found = get_current_variables()['not_found']
    firstpage_urls = get_current_variables()['firstpage_urls']
    newspaper_urls = get_current_variables()['newspaper_urls']
    page_urls = get_current_variables()['page_urls']
    ambiguous = get_current_variables()['ambiguous']
    no_link = get_current_variables()['no_link']
    
    if newspaper.startswith("*") or newspaper.startswith("-") or 'date unknown' in newspaper:
        # print('--> skipping (due to invalid name for newspaper)')
        continue

        
    date_as_string = f'{date[0]}-{date[1]}-{date[2]}'
    
    
    try:
        print(no_link[newspaper_clean][date_as_string])
        print('it is true so skip!')
        continue
    except:
        pass
    
        
    if not page:
        print('--> skipping (due to missing page number)')
        continue
        
        
    try:
        page_urls[newspaper_clean][date_as_string][page]
        print('--> skipping (due to already done)')
        continue
    except:
        pass
    
    
    if int(date[0]) < 1930 or int(date[0]) > 1939:
        print('--> skipping (due to date out of range)')
        continue
        
        
        
    skip_ahead = False
    
    newspaper_clean = re.sub(r' [A-Z][A-Z] ', ' ', newspaper)
    
    if newspaper_clean in not_found or newspaper_clean in ambiguous:
        # print('newspaper cannot be found', newspaper_clean)
        continue
    
    print()
    print("PROCESSING", newspaper_clean, f' {date[0]}-{date[1]}-{date[2]} ({counter}/{count})')
    
    if not newspaper_clean in newspaper_urls or newspaper_urls[newspaper_clean] == '':
        newspaper_urls[newspaper_clean] = ''
        if newspaper_urls[newspaper_clean] == '' or '#containing=' in newspaper_urls[newspaper_clean] or '/browse' in newspaper_urls[newspaper_clean]:
            print('searching for newspaper', newspaper_clean)
            b.get('https://www.newspapers.com/papers/')
            time.sleep(3)
            b.find_element_by_css_selector('[name="filtertext"]').send_keys(newspaper_clean)
            time.sleep(1)
            b.find_element_by_css_selector('[name="filtertext"]').send_keys(Keys.ENTER)
            time.sleep(5)
            if b.find_element_by_css_selector("#noresults"):
                if 'nothing matched your search' in b.find_element_by_css_selector("#noresults").text:
                    print('could not find newspaper', newspaper_clean, '-- searched for', newspaper_clean)
                    not_found.append(newspaper_clean)
                    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
                    skip_ahead = True
            if b.find_elements_by_css_selector('.record-result'):
                if len(b.find_elements_by_css_selector('.record-result')) == 1:
                    b.find_elements_by_css_selector('.record-result')[0]
                    b.find_elements_by_css_selector('.record-result')[0].find_elements_by_tag_name('a')[0].click()
                    time.sleep(3)
                    newspaper_url = b.current_url
                    newspaper_urls[newspaper_clean] = newspaper_url
                else:
                    test = [x for x in b.find_elements_by_css_selector('.record-result') if x.find_element_by_tag_name('h2').text == newspaper_clean]
                    if len(test) == 1:
                        test[0].find_elements_by_tag_name('a')[0].click()
                        time.sleep(3)
                        newspaper_url = b.current_url
                        newspaper_urls[newspaper_clean] = newspaper_url                        
                    else:
                        ambiguous.append(newspaper_clean)
                        set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
                        print('--> skipping (due to ambivalent newspaper name)')
                        skip_ahead = True

    
    
    if skip_ahead:
        continue
        
    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
    not_found = get_current_variables()['not_found']
    firstpage_urls = get_current_variables()['firstpage_urls']
    newspaper_urls = get_current_variables()['newspaper_urls']
    page_urls = get_current_variables()['page_urls']
    ambiguous = get_current_variables()['ambiguous']
    no_link = get_current_variables()['no_link']

    
    if not newspaper_clean in firstpage_urls:
        firstpage_urls[newspaper_clean] = {}
    
    if not date_as_string in firstpage_urls[newspaper_clean] or not '/image/' in firstpage_urls[newspaper_clean][date_as_string]:
        print(f'have browse page for {newspaper_clean} -- looking for first page for date')
        b.get(newspaper_urls[newspaper_clean])
        time.sleep(3)

        try:
            # choose year
            b.find_element_by_id('datepicker_year_combobox').send_keys(date[0])
            time.sleep(3)
        except:
            time.sleep(3)
            b.find_element_by_id('datepicker_year_combobox').send_keys(date[0])
            time.sleep(3)

        # choose month
        select = Select(b.find_element_by_id('select_month'))
        select.select_by_visible_text(date[1])
        time.sleep(3)

        # choose day
        day_element = [x for x in b.find_elements_by_class_name('calendar_option') if x.text == date[2]][0]
        try:
            day_element.find_element_by_tag_name('a')
            day_element.click()
            time.sleep(3)

            if not date_as_string in firstpage_urls[newspaper_clean]:
                firstpage_urls[newspaper_clean][date_as_string] = b.current_url
        except NoSuchElementException:
            if not newspaper_clean in no_link:
                no_link[newspaper_clean] = {}
            if not date_as_string in no_link[newspaper_clean]:
                no_link[newspaper_clean][date_as_string] = True
            print('--> skipping (due to missing date)')
            skip_ahead = True
    
    
    if skip_ahead:
        continue
        

    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
    not_found = get_current_variables()['not_found']
    firstpage_urls = get_current_variables()['firstpage_urls']
    newspaper_urls = get_current_variables()['newspaper_urls']
    page_urls = get_current_variables()['page_urls']
    ambiguous = get_current_variables()['ambiguous']
    no_link = get_current_variables()['no_link']

    page = page.groups()[0]
    if not newspaper_clean in page_urls:
        page_urls[newspaper_clean] = {}
    
    if not date_as_string in page_urls[newspaper_clean]:
        page_urls[newspaper_clean][date_as_string] = {}
        
    if not page in page_urls[newspaper_clean][date_as_string]:
        if firstpage_urls[newspaper_clean][date_as_string].startswith('https://www.newspapers.com/paper'):
            skip_ahead = True
            del firstpage_urls[newspaper_clean][date_as_string]
            set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
        else:
            print(f'finding page {page} for date {date_as_string} in {newspaper_clean} -- browsing from first page')

            b.get(firstpage_urls[newspaper_clean][date_as_string])
            time.sleep(2)

            if int(page) > 1:
                print(f'setting key to {int(page)-1}')
                b.find_element_by_id('filmstrip_pagenum_target_input').send_keys(int(page)-1)
                b.find_element_by_id('filmstrip_pagenum_target_input').send_keys(Keys.ENTER)
            
            max_count = int(b.find_element_by_id('filmstrip_total_edition_count').text)
            if int(max_count) > int(page):
                continue
            else:
                current = b.find_element_by_id('filmstrip_pagenum_target_input').get_attribute('value')
                while not int(current) == int(page):
                    b.find_element_by_css_selector('a[title="Next page"]').click()
                    time.sleep(1)
                    current = int(b.find_element_by_id('filmstrip_pagenum_target_input').get_attribute('value'))

                page_url = b.current_url

                if not newspaper_clean in page_urls:
                    page_urls[newspaper_clean] = {}

                if not date_as_string in page_urls[newspaper_clean]:
                    page_urls[newspaper_clean][date_as_string] = {}

                page_urls[newspaper_clean][date_as_string][page] = page_url

    
    if skip_ahead:
        continue
        
    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)

    print("DONE", newspaper_clean, '-->', page_urls[newspaper_clean][date_as_string][page])
    print()
    


PROCESSING Aiken Standard  1934-January-24 (7/3796)
DONE Aiken Standard --> https://www.newspapers.com/image/14191747


PROCESSING Akron Beacon Journal  1937-April-13 (8/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228680699


PROCESSING Akron Beacon Journal  1930-August-1 (9/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228836625


PROCESSING Akron Beacon Journal  1937-August-31 (10/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228732511


PROCESSING Akron Beacon Journal  1935-December-11 (11/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228473631


PROCESSING Akron Beacon Journal  1935-December-12 (12/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228474387


PROCESSING Akron Beacon Journal  1935-December-14 (13/3796)
DONE Akron Beacon Journal --> https://www.newspapers.com/image/228475598


PROCESSING Akron Beacon Journal  1935-December-16 (14/3796)
DONE Akron Beacon J

--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)

PROCESSING Arizona Republic  1936-January-24 (212/3796)
DONE Arizona Republic --> https://www.newspapers.com/image/117186988


PROCESSING Arizona Republic  1936-May-29 (213/3796)
DONE Arizona Republic --> https://www.newspapers.com/image/116955128


PROCESSING Arizona Republic  1935-October-16 (214/3796)
DONE Arizona Republic --> https://www.newspapers.com/image/117166064


PROCESSING Asbury Park Press  1934-August-3 (215/3796)
DONE Asbury Park Press --> https://www.newspapers.com/image/143319972


PROCESSING Asbury Park Press  1934-August-4 (216/3796)
DONE Asbury Park Press --> https://www.newspapers.com/image/143319993


PROCESSING Asbury Park Press  1937-February-5 (217/3796)
DONE Asbury Park Press --> https://www.newspapers.com/image/143340946


PROCESSING Asbury Park Press 

--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to date out of range)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping (due to date out of range)
--> skipping (due to missing page number)
--> skipping (due to missing page number)
--> skipping


PROCESSING Boston Globe  1933-April-17 (770/3796)
DONE Boston Globe --> https://www.newspapers.com/image/431071868


PROCESSING Boston Globe  1933-April-19 (771/3796)
DONE Boston Globe --> https://www.newspapers.com/image/431076440


PROCESSING Boston Globe  1936-April-22 (772/3796)
DONE Boston Globe --> https://www.newspapers.com/image/432014398


PROCESSING Boston Globe  1930-August-25 (773/3796)
DONE Boston Globe --> https://www.newspapers.com/image/430563426


PROCESSING Boston Globe  1930-August-26 (774/3796)
DONE Boston Globe --> https://www.newspapers.com/image/430563694


PROCESSING Boston Globe  1930-August-27 (775/3796)
DONE Boston Globe --> https://www.newspapers.com/image/430564084


PROCESSING Boston Globe  1931-July-20 (776/3796)
DONE Boston Globe --> https://www.newspapers.com/image/431169656


PROCESSING Boston Globe  1931-July-21 (777/3796)
DONE Boston Globe --> https://www.newspapers.com/image/431172019


PROCESSING Boston Globe  1931-July-24 (778/3796)
DONE Boston G

--> skipping (due to date out of range)
--> skipping (due to missing page number)

PROCESSING Brooklyn Daily Eagle  1932-May-18 (868/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/59971823

--> skipping (due to missing page number)

PROCESSING Brooklyn Daily Eagle  1934-May-19 (870/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/58260824

--> skipping (due to missing page number)

PROCESSING Brooklyn Daily Eagle  1936-May-20 (872/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/577442397

--> skipping (due to missing page number)

PROCESSING Brooklyn Daily Eagle  1934-November-1 (874/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/59971096


PROCESSING Brooklyn Daily Eagle  1939-November-10 (875/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/52759335


PROCESSING Brooklyn Daily Eagle  1938-November-11 (876/3796)
DONE Brooklyn Daily Eagle --> https://www.newspapers.com/image/5262190

'10'

In [13]:
'''
Amsterdam Evening Recorder
Baldwinsville Gazette and Farmers Journal
'''
pass

In [102]:
def get_newspapers(df, start_year=1930, end_year=1940):
    remaining = []
    newspaper_urls = get_current_variables()['newspaper_urls']
    has_content = [x for x in newspaper_urls if newspaper_urls]
    for newspaper, row in df.groupby('Newspaper'):
        newspaper = re.sub(r' [A-Z][A-Z] ', ' ', newspaper)
        if newspaper.startswith('*') or newspaper == "":
            continue
            
        valid = 1930 > int(min([x[:4] for x in row.Date]))
        valid = 1940 < int(max([x[:4] for x in row.Date]))
        
        if not valid:
            continue

        if newspaper in has_content:
            continue
            
        has_articles_in_range = [x for x in row['Source clean'] if not re.search('\d{6,10}', x) and ('1930' in x or '1931' in x or '1932' in x or '1933' in x or '1934' in x or '1935' in x or '1936' in x or '1937' in x or '1938' in x or '1939' in x)]

        if not has_articles_in_range:
            continue
            
        remaining.append(newspaper)
    
    return remaining
    
get_newspapers(df)

['Baltimore Afro-American',
 'Billboard',
 'Binghamton Press and Sun-Bulletin',
 'Boston Globe',
 'Chicago Defender',
 'Cincinnati Enquirer',
 'Daily News',
 'Daily Oklahoman',
 'Dayton Daily News',
 'Dayton Herald',
 'Des Moines Register',
 'Des Moines Tribune',
 'Detroit Free Press',
 'Dunkirk Evening Observer',
 'Elmira Star-Gazette',
 'Evening Sun',
 'Glens Falls Post-Star',
 'Miami Daily News',
 'Oakland Tribune',
 'San Antonio Light',
 'Washington Evening Star']

In [112]:
mapping = {
    'Alexandria Town Talk': 'The Town Talk\nAlexandria, Louisiana',
    'Binghamton Press and Sun-Bulletin': 'Press and Sun-Bulletin\nBinghamton, New York',
    "Boston Globe": "The Boston Globe\nBoston, Massachusetts",
    "Cincinnati Enquirer": "The Cincinnati Enquirer",
    "Daily Oklahoman": "The Daily Oklahoman",
    "Dayton Daily News": "Dayton Daily News\nDayton, Ohio",
    "Dayton Herald": "The Dayton Herald\nDayton, Ohio",
    "Des Moines Register": "The Des Moines Register\nDes Moines, Iowa",
    "Des Moines Tribune": "Des Moines Tribune\nDes Moines, Iowa",
    "Detroit Free Press": "Detroit Free Press\nDetroit, Michigan",
    "Dunkirk Evening Observer": "Dunkirk Evening Observer\nDunkirk, New York",
    "Elmira Star-Gazette": "Star-Gazette\nElmira, New York",
    "Glens Falls Post-Star": "The Post-Star\nGlens Falls, New York",
    "Oakland Tribune": "Oakland Tribune\nOakland, California",
    "San Antonio Light": "San Antonio Light\nSan Antonio, Texas",
    "Washington Evening Star": "Evening Star\nWashington, District of Columbia"
}

In [123]:
excluded = ['Baltimore Afro-American', 'Chicago Defender', 'Billboard', 'Hollywood Reporter', 'Daily News', 'Evening Sun']

In [116]:
newspapers = sorted([re.sub(' [A-Z][A-Z] ',' ',x) for x in set(df['Newspaper']) if x and re.sub(' [A-Z][A-Z] ',' ',x) not in firstpage_urls.keys() and not x.startswith('*')])

In [132]:
for newspaper in get_newspapers(df):
    not_found = get_current_variables()['not_found']
    firstpage_urls = get_current_variables()['firstpage_urls']
    newspaper_urls = get_current_variables()['newspaper_urls']
    page_urls = get_current_variables()['page_urls']
    ambiguous = get_current_variables()['ambiguous']
    no_link = get_current_variables()['no_link']

    if newspaper in excluded:
        continue
        
    if not newspaper in newspaper_urls or newspaper_urls[newspaper_clean] == '':
        newspaper_urls[newspaper_clean] = ''
        if newspaper_urls[newspaper_clean] == '' or '#containing=' in newspaper_urls[newspaper_clean] or '/browse' in newspaper_urls[newspaper_clean]:
            print('searching for newspaper', newspaper)
            b.get('https://www.newspapers.com/papers/')
            time.sleep(3)
            b.find_element_by_css_selector('[name="filtertext"]').send_keys(newspaper)
            time.sleep(1)
            b.find_element_by_css_selector('[name="filtertext"]').send_keys(Keys.ENTER)
            time.sleep(3)

            if b.find_element_by_css_selector("#noresults"):
                if 'nothing matched your search' in b.find_element_by_css_selector("#noresults").text:
                    print('could not find newspaper', newspaper_clean, '-- searched for', newspaper_clean)
                    not_found.append(newspaper_clean)
                    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
                    skip_ahead = True
            if b.find_elements_by_css_selector('.record-result'):
                if len(b.find_elements_by_css_selector('.record-result')) == 1:
                    b.find_elements_by_css_selector('.record-result')[0]
                    b.find_elements_by_css_selector('.record-result')[0].find_elements_by_tag_name('a')[0].click()
                    time.sleep(3)
                    newspaper_url = b.current_url
                    newspaper_urls[newspaper_clean] = newspaper_url
                else:
                    test = [x for x in b.find_elements_by_css_selector('.record-result') if x.find_element_by_tag_name('h2').text == newspaper_clean]
                    if len(test) == 1:
                        test[0].find_elements_by_tag_name('a')[0].click()
                        time.sleep(3)
                        newspaper_url = b.current_url
                        newspaper_urls[newspaper_clean] = newspaper_url                        
                    else:
                        test = [x for x in b.find_elements_by_css_selector('.record-result') if mapping[newspaper] in x.text]
                        if len(test) == 1:
                            test[0].find_elements_by_tag_name('a')[0].click()
                            time.sleep(3)
                            newspaper_url = b.current_url
                            newspaper_urls[newspaper_clean] = newspaper_url                        
                        else:
                            ambiguous.append(newspaper_clean)
                            set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)
                            print('--> skipping (due to ambivalent newspaper name)')
                            skip_ahead = True


    set_current_variables(not_found, firstpage_urls, newspaper_urls, page_urls, ambiguous, no_link)


searching for newspaper Miami Daily News


NoSuchWindowException: Message: Browsing context has been discarded


In [122]:
sorted([x for x in newspaper_urls.keys()])

['Aiken Standard',
 'Akron Beacon Journal',
 'Albuquerque Journal',
 'Allentown Morning Call',
 'Alton Illinois Evening Telegraph',
 'American Israelite',
 'Arizona Republic',
 'Asbury Park Press',
 'Atlanta Constitution',
 'Bakersfield Californian',
 'Bakersfield Morning Echo',
 'Barnard Bulletin',
 'Benton Harbor News-Palladium',
 'Berwyn Life',
 'Brooklyn Daily Eagle',
 'Camden Courier-Post',
 'Hackensack Record',
 'Harlingen Valley Morning Star',
 'Harrisburg Sunday Courier',
 'Harrisburg Telegraph',
 'Hartford Courant',
 'Hazleton Plain Speaker',
 'Hazleton Standard-Speaker',
 'Helper Journal',
 'Herald-Journal',
 'Herald-Press',
 'Hollywood Reporter',
 'Honolulu Star-Bulletin',
 'Huntsville Times',
 'Idaho Falls Post-Register',
 'Independent-Record',
 'Indianapolis News',
 'Indianapolis Star',
 'Indianapolis Times',
 'Ithaca Journal',
 'Ithaca Journal-News',
 'Jackson Clarion-Ledger',
 'Johnson City Chronicle',
 'Johnson City Staff-News',
 'Journal',
 'Journal News',
 'Journal Ti

In [ ]:
get_current_variables()['newspaper_urls']

['The Town Talk\nAlexandria, Louisiana\n1,723,513 pages']

In [ ]:
"Binghamton Press and Sun-Bulletin": "https://www.newspapers.com/paper/press-and-sun-bulletin/3799/",
"Boston Globe": "https://www.newspapers.com/paper/the-boston-globe/9077/",
"Cincinnati Enquirer": "https://www.newspapers.com/paper/the-cincinnati-enquirer/844/",
"Daily Oklahoman": "https://www.newspapers.com/paper/the-daily-oklahoman/2087/",
"Dayton Daily News": "https://www.newspapers.com/paper/dayton-daily-news/7809/",
"Dayton Herald": "https://www.newspapers.com/paper/the-dayton-herald/7803/",
"Des Moines Register": "https://www.newspapers.com/paper/the-des-moines-register/86/",
"Des Moines Tribute": "https://www.newspapers.com/paper/des-moines-tribune/84/",
"Detroit Free Press": "https://www.newspapers.com/paper/detroit-free-press/3676/",
"Dunkirk Evening Observer": "https://www.newspapers.com/paper/dunkirk-evening-observer/500/",
"Elmira Star-Gazette": "https://www.newspapers.com/paper/star-gazette/3800/",
"Oakland Tribune": "https://www.newspapers.com/paper/oakland-tribune/2/",
"San Antonio Light": "https://www.newspapers.com/paper/san-antonio-light/607/",
"Washington Evening Star": "https://www.newspapers.com/paper/evening-star/1353/",

In [ ]:
Glens Falls Post-Star
Lima News